## Intro

This is inspired by 
Article (likas2001probability) Likas, A. Probability density estimation using artificial neural networks Computer physics communications, Elsevier, 2001, 135, 167-175

But rather than estimating the working with a network, we will instead work with its derivitive.
This will let us replace their integration with a derivative.

Note that this method only works for compact supports



they use the PDF is given by $$p_h(x,p) = \dfrac{h(x,p)}{\int_S h(z,p) dz}$$
and in their case $h=N(x,p)$  a neural network with weight and bias parameters $p$.
Where $S$ is a compact support. (That means bounded)


But if instead we say $h=\frac{\partial N(x,p)}{\partial x}$,

then $$p_h(x,p) = \dfrac{h(x,p)}{\int_S h(z,p)}=\dfrac{\frac{\partial N(x,p)}{\partial x}}{N(max(S),p) - N(min(S), p)}$$

The denominator is ofcourse more complex for non-1D values of S.


The loss function given is the negative log-likelihood of the set of training samples $X$
$$L(p) = -\sum_\forall{x \in X} ln(h(x,p))  + |X| ln(\int_S h(z,p) dx)$$

Which befomes:

$$L(p) = -\sum_\forall{x \in X} log(\frac{\partial N(x,p)}{\partial x})  + |X| ln(\int_S h(z,p)) dx$$

In [7]:
using StatsBase
using Distributions

In [2]:
using TensorFlow
using MLDataUtils

In [3]:
using DensityEstimationML
using Plots

using MacroTools
macro plot(ex)
    @capture(ex, (x_, [ys__], tail__ )) 
    labels = repr.(ys)
    ys_expr = Expr(:hvect, ys...)
    labels_expr = Expr(:vect, repr.(ys)...)
    Expr(:call, :plot, x, ys_expr, tail..., Expr(:kw, :labels, labels_expr))
end

@plot (X, [Z, logistic.(Z)])

In [4]:
immutable NeuralDensityEstimator
    sess::Session
    
    #Network nodes
    optimizer::Tensor
    t::Tensor
    pdf::Tensor
    cdf::Tensor
end

In [5]:
abstract type NdeModel; end
immutable NdeNaive <: NdeModel; end
immutable NdeAsymptoticForced <: NdeModel; end

In [ ]:
leaky_relu6(z) = 0.001z + nn.relu6(z)

NeuralDensityEstimator(prob_layer_sizes) = NeuralDensityEstimator(prob_layer_sizes, NdeNaive())

function NeuralDensityEstimator(prob_layer_sizes, model::NdeNaive)
    sess = Session(Graph())
    @tf begin
        t1 = placeholder(Float32, shape=[1, -1])
        t2 = placeholder(Float32, shape=[1, -1])
        countprob_between = placeholder(Float32, shape=[-1])
        
        zp1 = [t1]
        zp2 = [t2]
        layer_sizes= [1; prob_layer_sizes; 1]
        act_funs = Vector{Function}(length(layer_sizes)-1)
        act_funs[:] = nn.relu6
        act_funs[end] = nn.sigmoid
        for ii in 2:length(layer_sizes)
            below_size = layer_sizes[ii-1]
            above_size = layer_sizes[ii]
            
            act_fun = act_funs[ii-1]
            
            Wii = get_variable("W_$ii", [above_size, below_size], Float32)
            bii = get_variable("b_$ii", [above_size, 1], Float32)
            
            push!(zp1, act_fun(Wii*zp1[end] .+ bii))
            push!(zp2, act_fun(Wii*zp2[end] .+ bii))
        end
        
        cdf1 = zp1[end]
        cdf2 = zp2[end]
        #Assumes t2>t1
        cdf_between = cdf2 - cdf1
                
        
        losses = 0.5.*(cdf_between .- countprob_between).^2
        
        loss=reduce_mean(losses; axis=2)
        optimizer = train.minimize(train.AdamOptimizer(), loss)
        
        pdf1 = gradients(cdf1, t1)
    end
    
    run(sess, global_variables_initializer())
    
    NeuralDensityEstimator(sess, optimizer, t1, pdf1, cdf1)
end

In [ ]:
tanh.(-1000:1000)

In [ ]:
plot(x->logistic(x*logistic(10x-2)), xlims=(-10, 10))

In [ ]:
function NeuralDensityEstimator(prob_layer_sizes, model::NdeAsymptoticForced)
    sess = Session(Graph())
    @tf begin
        t1 = placeholder(Float32, shape=[1, -1])
        t2 = placeholder(Float32, shape=[1, -1])
        countprob_between = placeholder(Float32, shape=[-1])
        
        zp1 = [t1]
        zp2 = [t2]
        layer_sizes= [1; prob_layer_sizes; 1]
        act_funs = Vector{Function}(length(layer_sizes)-1)
        act_funs[:] = nn.relu6
        act_funs[end] = TensorFlow.identity
        for ii in 2:length(layer_sizes)
            below_size = layer_sizes[ii-1]
            above_size = layer_sizes[ii]
            
            act_fun = act_funs[ii-1]
            
            Wii = get_variable("W_$ii", [above_size, below_size], Float32)
            bii = get_variable("b_$ii", [above_size, 1], Float32)
            
            push!(zp1, act_fun(Wii*zp1[end] .+ bii))
            push!(zp2, act_fun(Wii*zp2[end] .+ bii))
        end
        
        #forces assymtoptess    
        s_shift = get_variable([1], Float32) #support translate
        s_scale = get_variable([1], Float32) #support scale
        cft_fun(z, t) = nn.sigmoid(2(s_scale*t + s_shift)) + tanh(s_scale*t + s_shift)^2 * z
                    
        cdf1 = cft_fun(zp1[end], t1)
        cdf2 = cft_fun(zp2[end], t2)
        #Assumes t2>t1
        cdf_between = cdf2 - cdf1
                
        
        losses = 0.5.*(cdf_between .- countprob_between).^2
        
        loss=reduce_mean(losses; axis=2)
        optimizer = train.minimize(train.AdamOptimizer(), loss)
        
        pdf1 = gradients(cdf1, t1)
    end
    
    run(sess, global_variables_initializer())
    
    NeuralDensityEstimator(sess, optimizer, t1, pdf1, cdf1)
end

In [8]:
using StatsFuns

In [75]:
X=-2:0.01:2
Z=(0.4X-tanh.(X).^2)
@plot (X, [Z, logistic.(Z), Z.*Z, logistic.(Z.*Z)])

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] (::Base.##716#717)(::Array{Float64,1}, ::Array{Float64,1}) at ./deprecated.jl:346
 [3] include_string(::String, ::String) at ./loading.jl:515
 [4] include_string(::Module, ::String, ::String) at /home/uniwa/students2/students/20361362/linux/.julia/v0.6/Compat/src/Compat.jl:577
 [5] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /home/uniwa/students2/students/20361362/linux/.julia/v0.6/IJulia/src/execute_request.jl:154
 [6] eventloop(::ZMQ.Socket) at /home/uniwa/students2/students/20361362/linux/.julia/v0.6/IJulia/src/eventloop.jl:8
 [7] (::IJulia.##14#17)() at ./task.jl:335
while loading In[75], in expression starting on line 3
Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] (::Base.##716#717)(::Array{Float64,1}, ::Array{Float64,1}) at ./deprecated.jl:346
 [3] include_string(::String, ::String) at ./loading.jl:515
 [4] include_string(::Module, ::String, ::String) at /home/uniwa/students2/stu

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,376.378 40.752,375.958 42.1339,375.537 43.5157,375.113 44.8976,374.687 46.2795,374.259 47.6614,373.829 49.0433,373.397 50.4252,372.963 51.8071,372.526 
 53.189,372.087 54.5709,371.646 55.9528,371.202 57.3346,370.756 58.7165,370.308 60.0984,369.857 61.4803,369.403 62.8622,368.947 64.2441,368.488 65.626,368.027 
 67.0079,367.563 68.3898,367.096 69.7717,366.627 71.1535,366.154 72.5354,365.679 73.9173,365.201 75.2992,364.72 76.6811,364.236 78.063,363.749 79.4449,363.259 
 80.8268,362.765 82.2087,362.269 83.5906,361.769 84.9724,361.266 86.3543,360.76 87.7362,360.25 89.1181,359.737 90.5,359.221 91.8819,358.701 93.2638,358.177 
 94.6457,357.65 96.0276,357.119 97.4094,356.585 98.7913,356.047 100.173,355.505 101.555,354.959 102.937,354.409 104.319,353.856 105.701,353.298 107.083,352.737 
 108.465,352.171 109.846,351.601 111.228,351.027 112.61,350.449 113.992,349.867 115.374,349.28 116.756,348.689 118.138,348.094 119.52,347.494 120.902,346.89 
 122.283,346.281 123.665,345.667 125.047,345.049 126.429,344.427 127.811,343.799 129.193,343.167 130.575,342.53 131.957,341.889 133.339,341.242 134.72,340.591 
 136.102,339.935 137.484,339.274 138.866,338.608 140.248,337.937 141.63,337.26 143.012,336.579 144.394,335.893 145.776,335.202 147.157,334.505 148.539,333.804 
 149.921,333.097 151.303,332.385 152.685,331.668 154.067,330.945 155.449,330.218 156.831,329.485 158.213,328.747 159.594,328.004 160.976,327.256 162.358,326.502 
 163.74,325.744 165.122,324.98 166.504,324.211 167.886,323.437 169.268,322.657 170.65,321.873 172.031,321.084 173.413,320.289 174.795,319.49 176.177,318.685 
 177.559,317.876 178.941,317.062 180.323,316.243 181.705,315.419 183.087,314.591 184.469,313.758 185.85,312.92 187.232,312.079 188.614,311.232 189.996,310.382 
 191.378,309.527 192.76,308.668 194.142,307.805 195.524,306.938 196.906,306.067 198.287,305.193 199.669,304.315 201.051,303.434 202.433,302.55 203.815,301.662 
 205.197,300.772 206.579,299.878 207.961,298.982 209.343,298.084 210.724,297.183 212.106,296.28 213.488,295.375 214.87,294.469 216.252,293.56 217.634,292.651 
 219.016,291.74 220.398,290.828 221.78,289.916 223.161,289.003 224.543,288.09 225.925,287.176 227.307,286.263 228.689,285.351 230.071,284.439 231.453,283.528 
 232.835,282.618 234.217,281.71 235.598,280.803 236.98,279.899 238.362,278.997 239.744,278.098 241.126,277.201 242.508,276.308 243.89,275.418 245.272,274.532 
 246.654,273.65 248.035,272.772 249.417,271.9 250.799,271.032 252.181,270.169 253.563,269.313 254.945,268.462 256.327,267.617 257.709,266.78 259.091,265.949 
 260.472,265.125 261.854,264.309 263.236,263.501 264.618,262.701 266,261.909 267.382,261.127 268.764,260.353 270.146,259.589 271.528,258.835 272.909,258.091 
 274.291,257.357 275.673,256.634 277.055,255.921 278.437,255.22 279.819,254.531 281.201,253.853 282.583,253.187 283.965,252.534 285.346,251.893 286.728,251.265 
 288.11,250.65 289.492,250.048 290.874,249.46 292.256,248.885 293.638,248.324 295.02,247.778 296.402,247.245 297.783,246.727 299.165,246.224 300.547,245.735 
 301.929,245.261 303.311,244.802 304.693,244.359 306.075,243.93 307.457,243.517 308.839,243.119 310.22,242.737 311.602,242.37 312.984,242.018 314.366,241.683 
 315.748,241.363 317.13,241.058 318.512,240.769 319.894,240.496 321.276,240.238 322.657,239.996 324.039,239.769 325.421,239.558 326.803,239.362 328.185,239.181 
 329.567,239.015 330.949,238.865 332.331,238.729 333.713,238.608 335.094,238.501 336.476,238.409 337.858,238.331 339.24,238.267 340.622,238.217 342.004,238.181 
 343.386,238.158 344.768,238.149 346.15,238.152 347.531,238.169 348.913,238.197 350.295,238.239 351.677,238.292 353.059,238.357 354.441,238.433 355.823,238.521 
 357.205,238.619 358.587,23

In [83]:
plot(X,log(cosh(X)))


Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] cosh(::StepRangeLen{Float64,Base.TwicePrecision{Float64},Base.TwicePrecision{Float64}}) at ./deprecated.jl:57
 [3] include_string(::String, ::String) at ./loading.jl:515
 [4] include_string(::Module, ::String, ::String) at /home/uniwa/students2/students/20361362/linux/.julia/v0.6/Compat/src/Compat.jl:577
 [5] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /home/uniwa/students2/students/20361362/linux/.julia/v0.6/IJulia/src/execute_request.jl:154
 [6] eventloop(::ZMQ.Socket) at /home/uniwa/students2/students/20361362/linux/.julia/v0.6/IJulia/src/eventloop.jl:8
 [7] (::IJulia.##14#17)() at ./task.jl:335
while loading In[83], in expression starting on line 1


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,7.87402 40.752,10.5541 42.1339,13.2323 43.5157,15.9083 44.8976,18.5823 46.2795,21.2542 47.6614,23.9239 49.0433,26.5914 50.4252,29.2567 51.8071,31.9196 
 53.189,34.5803 54.5709,37.2385 55.9528,39.8943 57.3346,42.5477 58.7165,45.1985 60.0984,47.8468 61.4803,50.4924 62.8622,53.1354 64.2441,55.7756 65.626,58.4131 
 67.0079,61.0477 68.3898,63.6795 69.7717,66.3083 71.1535,68.9342 72.5354,71.557 73.9173,74.1766 75.2992,76.7932 76.6811,79.4065 78.063,82.0165 79.4449,84.6231 
 80.8268,87.2264 82.2087,89.8261 83.5906,92.4224 84.9724,95.015 86.3543,97.6039 87.7362,100.189 89.1181,102.77 90.5,105.348 91.8819,107.922 93.2638,110.491 
 94.6457,113.056 96.0276,115.618 97.4094,118.175 98.7913,120.727 100.173,123.275 101.555,125.819 102.937,128.358 104.319,130.892 105.701,133.422 107.083,135.947 
 108.465,138.467 109.846,140.982 111.228,143.491 112.61,145.996 113.992,148.495 115.374,150.989 116.756,153.477 118.138,155.96 119.52,158.437 120.902,160.908 
 122.283,163.373 123.665,165.832 125.047,168.286 126.429,170.733 127.811,173.173 129.193,175.607 130.575,178.035 131.957,180.456 133.339,182.87 134.72,185.277 
 136.102,187.677 137.484,190.07 138.866,192.456 140.248,194.834 141.63,197.205 143.012,199.568 144.394,201.923 145.776,204.271 147.157,206.61 148.539,208.941 
 149.921,211.264 151.303,213.578 152.685,215.884 154.067,218.181 155.449,220.468 156.831,222.747 158.213,225.017 159.594,227.277 160.976,229.528 162.358,231.769 
 163.74,234.001 165.122,236.222 166.504,238.433 167.886,240.634 169.268,242.824 170.65,245.004 172.031,247.173 173.413,249.331 174.795,251.477 176.177,253.613 
 177.559,255.737 178.941,257.849 180.323,259.949 181.705,262.038 183.087,264.114 184.469,266.178 185.85,268.229 187.232,270.267 188.614,272.292 189.996,274.305 
 191.378,276.303 192.76,278.289 194.142,280.26 195.524,282.218 196.906,284.162 198.287,286.091 199.669,288.005 201.051,289.905 202.433,291.79 203.815,293.66 
 205.197,295.515 206.579,297.354 207.961,299.177 209.343,300.984 210.724,302.775 212.106,304.55 213.488,306.308 214.87,308.05 216.252,309.774 217.634,311.481 
 219.016,313.171 220.398,314.843 221.78,316.497 223.161,318.133 224.543,319.75 225.925,321.35 227.307,322.93 228.689,324.492 230.071,326.034 231.453,327.557 
 232.835,329.061 234.217,330.544 235.598,332.008 236.98,333.452 238.362,334.875 239.744,336.277 241.126,337.659 242.508,339.019 243.89,340.358 245.272,341.676 
 246.654,342.972 248.035,344.246 249.417,345.499 250.799,346.729 252.181,347.936 253.563,349.121 254.945,350.283 256.327,351.422 257.709,352.537 259.091,353.629 
 260.472,354.698 261.854,355.743 263.236,356.763 264.618,357.76 266,358.732 267.382,359.68 268.764,360.603 270.146,361.502 271.528,362.375 272.909,363.223 
 274.291,364.046 275.673,364.844 277.055,365.615 278.437,366.361 279.819,367.082 281.201,367.776 282.583,368.444 283.965,369.086 285.346,369.701 286.728,370.29 
 288.11,370.852 289.492,371.388 290.874,371.897 292.256,372.378 293.638,372.833 295.02,373.261 296.402,373.661 297.783,374.034 299.165,374.38 300.547,374.699 
 301.929,374.99 303.311,375.253 304.693,375.489 306.075,375.697 307.457,375.878 308.839,376.03 310.22,376.156 311.602,376.253 312.984,376.322 314.366,376.364 
 315.748,376.378 317.13,376.364 318.512,376.322 319.894,376.253 321.276,376.156 322.657,376.03 324.039,375.878 325.421,375.697 326.803,375.489 328.185,375.253 
 329.567,374.99 330.949,374.699 332.331,374.38 333.713,374.034 335.094,373.661 336.476,373.261 337.858,372.833 339.24,372.378 340.622,371.897 342.004,371.388 
 343.386,370.852 344.768,370.29 346.15,369.701 347.531,369.086 348.913,368.444 350.295,367.776 351.677,367.082 353.059,366.361 354.441,365.615 355.823,364.844 
 357.205,364.046 358.587,363.223 

In [79]:
Z

401-element Array{Float64,1}:
 -1.72935 
 -1.72397 
 -1.71857 
 -1.71315 
 -1.70769 
 -1.70221 
 -1.69671 
 -1.69117 
 -1.68561 
 -1.68001 
 -1.67439 
 -1.66874 
 -1.66306 
  ⋮       
 -0.156738
 -0.15439 
 -0.152013
 -0.149606
 -0.14717 
 -0.144706
 -0.142213
 -0.139694
 -0.137147
 -0.134574
 -0.131974
 -0.129349

In [ ]:
aforce(f,t) = (tanh(t)^2)*f(t)

plot(t->aforce(logistic, t))

In [ ]:
function Distributions.cdf(est::NeuralDensityEstimator, t::Real)
    gr = est.sess.graph
    ts = reshape([t], (1,1))
    run(est.sess, est.cdf, Dict(est.t=>ts))[1]
end

function Distributions.pdf(est::NeuralDensityEstimator, t::Real)
    gr = est.sess.graph
    ts = reshape([t], (1,1))
    run(est.sess, est.pdf, Dict(est.t=>ts))[1]
end



In [ ]:
"""
Returns 3 vectors.
A vector of start points \$t1\$
A vector of end points \$t2\$
A vector of the counts of elements \$ti\$ between them, such that \$\left{t_i \mid t1 \le ti < t2 \right}\$
"""
function get_cdf_training_pairs(observations)
    observations = sort(observations)
    
    t1s = Float32[]
    t2s = Float32[]
    probs = Float32[]
    for (ii, t1) in enumerate(observations)
        for jj in ii+1 : length(observations)
            t2 = observations[jj]
            count = jj - ii           
            push!(t1s, t1)
            push!(t2s, t2)
            push!(probs, count/length(observations))
        end
    end
    
    t1s, t2s, probs
end

In [ ]:
function StatsBase.fit!(estimator::NeuralDensityEstimator, observations;
    epochs = 20,
    batch_size = 1024)
    gr = estimator.sess.graph
    
    for ii in 1:epochs
        batch_losses = Float32[]
        for (t1_o, t2_o, probs_o) = eachbatch(shuffleobs(get_cdf_training_pairs(observations)), batch_size)
            loss_o, _, = run(estimator.sess, 
                [gr["loss"], estimator.optimizer],
                Dict(gr["t1"]=>t1_o', gr["t2"]=>t2_o', gr["countprob_between"]=>probs_o))
            push!(batch_losses, loss_o[1])
        end
        epoch_loss = mean(batch_losses)
        println("Epoch $ii: loss: $(epoch_loss)")
    end
    estimator
end

In [ ]:
est = NeuralDensityEstimator([64, 64])
data = GenerateDatasets.magdon_ismail_and_atiya();


In [ ]:
fit!(est, data; epochs=100)

In [ ]:
histogram(data)

In [ ]:
X=-50:0.1:100

plot(X, pdf.(est, X))

In [ ]:
empirical_cdf(data, X) = [length(filter(i->i<x, data)) for x in X]./length(data)

In [ ]:
plot(X, [cdf.(est, X), empirical_cdf(data, X)] , label=["Estimated" "Empirical"])

In [ ]:
2π*sqrt(0.296/9.8)

In [ ]:
2π*sqrt(0.296/11.7)